In [ ]:
%matplotlib inline


# Searching on DARTS search space

In this tutorial, we demonstrate how to search on the famous model space proposed in [DARTS](https://arxiv.org/abs/1806.09055)_.

Through this process, you will learn:

* How to use the built-in model spaces from NNI's model space hub.
* How to use one-shot exploration strategies to explore a model space.
* How to customize evaluators to achieve the best performance.

In the end, we get a strong-performing model on CIFAR-10 dataset, which achieves xx.xx% accuracy.

.. attention::

   Running this tutorial requires a GPU.
   If you don't have one, you can set ``gpus`` in :class:`~nni.retiarii.evaluator.pytorch.Classification` to be 0,
   but do note that it will be much slower.

## Use the model space

The model space provided in DARTS originated from [NASNet](https://arxiv.org/abs/1707.07012)_,
where the full model is constructed by repeatedly stacking a single computational unit (called a **cell**).
There are two types of cells within a network. The first type is called *normal cell*, and the second type is called *reduction cell*.
The key difference between normal and reduction cell is that the reduction cell will downsample the input feature map,
and decrease its resolution. Normal and reduction cells are stacked alternately, as shown in the following figure.

<img src="file://../../img/nasnet_cell_stack.png">

A cell takes outputs from two previous cells as inputs and contains a collection of *nodes*.
Each node takes two previous nodes within the same cell (or the two cell inputs),
and applies an *operator* (e.g., convolution, or max-pooling) to each input,
and sums the outputs of operators as the output of the node.
The output of cell is the concatenation of all the nodes that are never used as inputs of another node.
We recommend reading [NDS](https://arxiv.org/pdf/1905.13214.pdf)_ or [ENAS](https://arxiv.org/abs/1802.03268)_ for details.

We illustrate an example of cells in the following figure.

<img src="file://../../img/nasnet_cell.png">

The search space proposed in DARTS paper introduced two modifications to the original space in [NASNet](https://arxiv.org/abs/1707.07012)_.

Firstly, the operator candidates have been narrowed down to seven:

- Max pooling 3x3
- Average pooling 3x3
- Skip connect (Identity)
- Separable convolution 3x3
- Separable convolution 5x5
- Dilated convolution 3x3
- Dilated convolution 5x5

Secondly, the output of cell is the concatenate of **all the nodes within the cell**.

As the search space is based on cell, once the normal and reduction cell has been fixed, we can stack them for indefinite times.
To save the search cost, the common practice is to reduce the number of filters (i.e., channels) and number of stacked cells
during the search phase, and increase them back when training the final searched architecture.

<div class="alert alert-info"><h4>Note</h4><p>DARTS is one of those papers that innovate both in search space and search strategy.
   In this tutorial, we will search on **model space** provided by DARTS with **search strategy** proposed by DARTS.
   We refer to them as *DARTS model space* (``DartsSpace``) and *DARTS strategy* (``DartsStrategy``), respectively.
   We did NOT imply that the DARTS space and DARTS strategy has to used together.
   You can always explore the DARTS space with another search strategy, or use your own strategy to search a different model space.</p></div>


In the following example, we initialize a DARTS model space, with only 16 initial filters and 8 stacked cells.
The network is specialized for CIFAR-10 dataset with 32x32 input resolution.

The DARTS model space here is provided by :doc:`model space hub <./space_hub>`,
where we have supported multiple popular model spaces for plug-and-play.

.. tip::

   The model space here can be replaced with any space provided in the hub,
   or even customized space built from scratch.



In [ ]:
from nni.retiarii.hub.pytorch import DARTS as DartsSpace

model_space = DartsSpace(16, 8, 'cifar')

## Search on the model space

To begin exploring the model space, one firstly need to have an evaluator to provide the criterion of a "good model".
As we are searching on CIFAR-10 dataset, one can easily use the :class:`~nni.retiarii.evaluator.pytorch.Classification`
as a starting point.

Note that for a typical setup of NAS, the model search should be on validation set, and the evaluation of the final searched model
should be on test set. However, as CIFAR-10 dataset only has a training set of 50k images and a validation set (10k images),
we have to split the original training set into a training set and a validation set.
As we are going to use the provided by DARTS paper, the recommended train/val split is 1:1.



In [ ]:
import nni
import numpy as np
from nni.retiarii.evaluator.pytorch import (
    Classification,
    DataLoader  # might also use torch.utils.data.DataLoader if not using multi-trial strategy
)
from torch.utils.data import SubsetRandomSampler
from torchvision import transforms
from torchvision.datasets import CIFAR10

CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
])

# If you are to use multi-trial strategies, please wrap CIFAR10 with :func:`nni.trace`.
train_data = nni.trace(CIFAR10)(root='./data', train=True, download=True, transform=transform)

num_samples = len(train_data)
indices = np.random.permutation(num_samples)
split = num_samples // 2

search_train_loader = DataLoader(
    train_data, batch_size=64, num_workers=6,
    sampler=SubsetRandomSampler(indices[:split]),
)

search_valid_loader = DataLoader(
    train_data, batch_size=64, num_workers=6,
    sampler=SubsetRandomSampler(indices[split:]),
)

<div class="alert alert-danger"><h4>Warning</h4><p>Please set ``fast_dev_run`` to False to reproduce the our claimed results.
   Otherwise, only a few mini-batches will be run.</p></div>



In [ ]:
fast_dev_run = True

evaluator = Classification(
    learning_rate=1e-3,
    weight_decay=1e-4,
    train_dataloaders=search_train_loader,
    val_dataloaders=search_valid_loader,
    max_epochs=10,
    gpus=1,
    fast_dev_run=fast_dev_run,
)

We will use DARTS (Differentiable ARchiTecture Search) as the search strategy to explore the model space.
DARTS strategy belongs to the category of `one-shot strategy <one-shot-nas>`.
The fundamental differences between One-shot strategies and `multi-trial strategies <multi-trial-nas>` is that,
one-shot strategy combines search with model training into a single run.
Compared to multi-trial strategies, one-shot NAS doesn't need to iteratively spawn new trials (i.e., models),
and thus saves the excessive cost of model training.
It's worth mentioning that one-shot NAS also suffers from multiple drawbacks despite its computational efficiency.
We recommend
[Weight-Sharing Neural Architecture Search: A Battle to Shrink the Optimization Gap](https://arxiv.org/abs/2008.01475)_
and
[How Does Supernet Help in Neural Architecture Search?](https://arxiv.org/abs/2010.08219)_ for interested readers.

If you want to know how DARTS strategy works, here is a brief version.
Under the hood, DARTS converts the cell into a densely connected graph, and put operators on edges (see the following figure).
Since the operators are not decided yet, every edge is a weighted mixture of multiple operators (multiple color in the figure).
DARTS then learns to assign the optimal "color" for each edge during the network training.
It finally selects one "color" for each edge, and drops redundant edges.
The weights on the edges are called *architecture weights*.

<img src="file://../../img/darts_illustration.png">

It's NOT reflected in the figure that, for DARTS model space, exactly two inputs are kept for every node.

:class:`~nni.retiarii.strategy.DARTS` strategy is provided as one of NNI's :doc:`built-in search strategies </nas/exploration_strategy>`.
Using it can be as simple as one line of code.



In [ ]:
from nni.retiarii.strategy import DARTS as DartsStrategy

strategy = DartsStrategy()

.. tip:: The ``DartsStrategy`` here can be replaced by any search strategies, even multi-trial strategies.

Launching the experiment is similar to what we have done in the :doc:`beginner tutorial <hello_nas>`,
except that the ``execution_engine`` argument should be set to ``oneshot``.



In [ ]:
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig

config = RetiariiExeConfig(execution_engine='oneshot')
experiment = RetiariiExperiment(model_space, evaluator=evaluator, strategy=strategy)
experiment.run(config)

.. tip::

   The search process can be visualized with tensorboard. For example::

       tensorboard --logdir=./lightning_logs

   Then, open the browser and go to http://localhost:6006/ to monitor the search process.

   .. image:: ../../img/darts_search_process.png

We can then retrieve the best model found by the strategy with ``export_top_models``.
Here, the retrieved model is a dict (called *architecture dict*) describing the selected normal cell and reduction cell.



In [ ]:
exported_arch = experiment.export_top_models()[0]

exported_arch

## Retrain the searched model

What we have got in the last step, is only a cell structure.
To get a final usable model with trained weights, we need to construct a real model based on this structure,
and then fully train it.

To construct a fixed model based on the architecture dict exported from the experiment,
we can use :func:`nni.retiarii.fixed_arch`. Seemingly, we are still creating a space.
But under the with-context, we are actually creating a fixed model.

Here, we increase the number of filters to 36, and number of cells to 20,
so as to reasonably increase the model size and boost the performance.



In [ ]:
from nni.retiarii import fixed_arch

with fixed_arch(exported_arch):
    final_model = DartsSpace(36, 20, 'cifar')

We then train the model on full CIFAR-10 training dataset, and evaluate it on the original CIFAR-10 validation dataset.



In [ ]:
train_loader = DataLoader(train_data, batch_size=96, num_workers=6)  # Use the original training data

transform_valid = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
])
valid_data = nni.trace(CIFAR10)(root='./data', train=False, download=True, transform=transform_valid)
valid_loader = DataLoader(train_data, batch_size=256, num_workers=6)

Create a new evaluator here because we can using a different data split.
Also, we should avoid the underlying pytorch-lightning implementation of Classification evaluator from loading the wrong checkpoint.



In [ ]:
max_epochs = 100

evaluator = Classification(
    learning_rate=1e-3,
    weight_decay=1e-4,
    train_dataloaders=train_loader,
    val_dataloaders=valid_loader,
    max_epochs=max_epochs,
    export_onnx=False,  # Disable ONNX export for this experiment
    fast_dev_run=fast_dev_run,  # Should be false for fully training
)

evaluator.fit(final_model)

When ``fast_dev_run`` is turned off, we get a model with the following architecture:

```python
{}
```
It achieves a validation accuracy of XXX%.

## Reproduce results in DARTS paper

You might notice there's still a gap between our results and the results in the DARTS paper.
This is because we didn't introduce some extra training tricks, including [DropPath](https://arxiv.org/pdf/1605.07648v4.pdf)_,
Auxiliary loss, gradient clipping and augmentations like [Cutout](https://arxiv.org/pdf/1708.04552v2.pdf)_.
They also train the networks for longer time (i.e., 600 epochs).

To implement these tricks, we need to rewrite a few parts of evaluator.

Working with one-shot strategies, evaluators need to be implemented in the style of `PyTorch-Lightning <lightning-evaluator>`,
The full tutorial can be found in :doc:`/nas/evaluator`.
Putting it briefly, the core part of writing a new evaluator is to write a new LightningModule.
[LightingModule](https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html)_ is a concept in
PyTorch-Lightning, which organizes the model training process into a list of functions, such as,
``training_step``, ``validation_step``, ``configure_optimizers``, etc.
Since we are merely adding a few ingredients to :class:`~nni.retiarii.evaluator.pytorch.Classification`,
we can simply inherit :class:`~nni.retiarii.evaluator.pytorch.ClassificationModule`, which is the underlying LightningModule
behind :class:`~nni.retiarii.evaluator.pytorch.Classification`.
This could look intimidating at first, but most of them are just plug-and-play tricks which you don't need to know details about.



In [ ]:
import torch
from nni.retiarii.evaluator.pytorch import ClassificationModule

class DartsClassificationModule(ClassificationModule):
    def __init__(
        self,
        learning_rate: float = 0.001,
        weight_decay: float = 0.,
        auxiliary_loss_weight: float = 0.4,
        max_epochs: int = 600
    ):
        self.auxiliary_loss_weight = auxiliary_loss_weight
        # Training length will be used in LR scheduler
        self.max_epochs = max_epochs
        super().__init__(learning_rate=learning_rate, weight_decay=weight_decay, export_onnx=False)

    def configure_optimizers(self):
        """Customized optimizer with momentum, as well as a scheduler."""
        optimizer = torch.optim.SGD(
            self.parameters(),
            momentum=0.9,
            lr=self.hparams.learning_rate,
            weight_decay=self.hparams.weight_decay
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, self.max_epochs, eta_min=1e-3)
        }

    def training_step(self, batch, batch_idx):
        """Training step, customized with auxiliary loss."""
        x, y = batch
        if self.auxiliary_loss_weight:
            y_hat, y_aux = self(x)
            loss_main = self.criterion(y_hat, y)
            loss_aux = self.criterion(y_aux, y)
            self.log('train_loss_main', loss_main)
            self.log('train_loss_aux', loss_aux)
            loss = loss_main + self.auxiliary_loss_weight * loss_aux
        else:
            y_hat = self(x)
            loss = self.criterion(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        for name, metric in self.metrics.items():
            self.log('train_' + name, metric(y_hat, y), prog_bar=True)
        return loss

    def on_train_epoch_start(self):
        # Set drop path probability before every epoch. This has no effect if drop path is not enabled in model.
        self.model.set_drop_path_prob(self.model.drop_path_prob * self.current_epoch / self.max_epochs)

        # Logging learning rate at the beginning of every epoch
        self.log('lr', self.trainer.optimizers[0].param_groups[0]['lr'])

The full evaluator is written as follows,
which simply wraps everything (except model space and search strategy of course), in a single object.
:class:`~nni.retiarii.evaluator.pytorch.Lightning` here is a special type of evaluator.
Don't forget to use the train/val data split specialized for search (1:1) here.



In [ ]:
from nni.retiarii.evaluator.pytorch import Lightning, Trainer

max_epochs = 50

evaluator = Lightning(
    DartsClassificationModule(0.025, 3e-4, 0., max_epochs),
    Trainer(
        gpus=1,
        max_epochs=max_epochs,
        fast_dev_run=fast_dev_run,
    ),
    train_dataloaders=search_train_loader,
    val_dataloaders=search_valid_loader
)

DARTS strategy is created with gradient clip turned on.
If you are familiar with PyTorch-Lightning, you might aware that gradient clipping can be enabled in Lightning trainer.
However, enabling gradient cip in the trainer above won't work, because the underlying
implementation of DARTS strategy is based on
[manual optimization](https://pytorch-lightning.readthedocs.io/en/stable/common/optimization.html)_.



In [ ]:
strategy = DartsStrategy(gradient_clip_val=5.)

Then we use the newly created evaluator and strategy to launch the experiment again.

<div class="alert alert-danger"><h4>Warning</h4><p>``model_space`` has to be re-instantiated because a known limitation,
   i.e., one model space can't be reused across multiple experiments.</p></div>



In [ ]:
model_space = DartsSpace(16, 8, 'cifar')

config = RetiariiExeConfig(execution_engine='oneshot')
experiment = RetiariiExperiment(model_space, evaluator=evaluator, strategy=strategy)
experiment.run(config)

exported_arch = experiment.export_top_models()[0]

exported_arch

When retraining,
we extend the original dataloader to introduce another trick called [Cutout](https://arxiv.org/pdf/1708.04552v2.pdf)_.
Cutout is a data augmentation technique that randomly masks out rectangular regions in images.
In CIFAR-10, the typical masked size is 16x16 (the image sizes are 32x32 in the dataset).



In [ ]:
def cutout_transform(img, length: int = 16):
    h, w = img.size(1), img.size(2)
    mask = np.ones((h, w), np.float32)
    y = np.random.randint(h)
    x = np.random.randint(w)

    y1 = np.clip(y - length // 2, 0, h)
    y2 = np.clip(y + length // 2, 0, h)
    x1 = np.clip(x - length // 2, 0, w)
    x2 = np.clip(x + length // 2, 0, w)

    mask[y1: y2, x1: x2] = 0.
    mask = torch.from_numpy(mask)
    mask = mask.expand_as(img)
    img *= mask
    return img

transform_with_cutout = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
    cutout_transform,
])

The train dataloader needs to be reinstantiated with the new transform.
The validation dataloader is not affected, and thus can be reused.



In [ ]:
train_data_cutout = nni.trace(CIFAR10)(root='./data', train=True, download=True, transform=transform_with_cutout)
train_loader_cutout = DataLoader(train_data_cutout, batch_size=96)

We then create the final model based on the new exported architecture.
This time, auxiliary loss and drop path probability is enabled.



In [ ]:
with fixed_arch(exported_arch):
    final_model = DartsSpace(36, 20, 'cifar', auxiliary_loss=True, drop_path_prob=0.2)

Launching the retraining requires creating another evaluator.
We can now put the gradient clipping in the keyword arguments of trainer.



In [ ]:
max_epochs = 600

evaluator = Lightning(
    DartsClassificationModule(0.025, 3e-4, 0.4, max_epochs),
    Trainer(
        gpus=1,
        gradient_clip_val=5.,
        max_epochs=max_epochs,
        fast_dev_run=fast_dev_run
    ),
    train_dataloaders=train_loader_cutout,
    val_dataloaders=valid_loader,
)

evaluator.fit(final_model)

<div class="alert alert-info"><h4>Note</h4><p>The full search and training takes around XX hours on a P100 GPU, and yields a top-1 accuracy of ~0.8%.</p></div>

